In [2]:
# We are using black as a code formatter
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
import requests
import json
import datetime
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import os
import re

<IPython.core.display.Javascript object>

In [4]:
# pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

<IPython.core.display.Javascript object>

Extraigamos los archivos parquet y los convertimos a dataframe

In [5]:
df = pd.read_parquet("../datasets/df_locationsWithAggregates.parquet")

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/df_locationsWithAggregates.parquet'

<IPython.core.display.Javascript object>

In [ ]:
df

In [ ]:
# guardar en una lista los nombres en formtato Iso3 de los países
iso3_code = [str(code) for code in df["Iso3"].values]
iso3_code

Maybe if we want to filter by WorldBankIncomeGroup (This step is not required.)

In [ ]:
df_high_income = df[(df["WorldBankIncomeGroup"] == "High-income countries")]
df_low_income = df[(df["WorldBankIncomeGroup"] == "Low-income countries")]
df_upper_middle_income = df[
    (df["WorldBankIncomeGroup"] == "Upper-middle-income countries")
]
df_lower_middle_income = df[
    (df["WorldBankIncomeGroup"] == "Lower-middle-income countries")
]

Revisamos una información básica de los dataframes

In [ ]:
directory = "../datasets/"
# Let's iterate each parquet file and get some basic information
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith(".parquet"):
        print("========" + filename.rstrip(".parquet").upper() + "========", end="\n")
        df = pd.read_parquet(f)
        df.replace("", float("NaN"), inplace=True)
        # print("====Basic info about the dataframe====", end="\n")
        # print(df.info(), end="\n")
        # print("====Sum the null in each column====", end="\n")
        # print(df.isnull().sum(), end="\n")
        # print("====Describe numeric columns====", end="\n")
        # print(df.describe(), end="\n")
        print("====Get a sample of 5 rows====", end="\n")
        print(df.sample(5), end="\n")
        print(end="\n")

Modificación de los nombres de las diferentes columnas

In [ ]:
directory = "../datasets/"
# Let's iterate each parquet file and get some basic information
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith(".parquet"):
        print("========" + filename.rstrip(".parquet").upper() + "========", end="\n")
        df = pd.read_parquet(f)
        df.replace("", float("NaN"), inplace=True)
        # columns_categories = df.columns
        print("====Basic info about the dataframe====", end="\n")
        print(df.columns)
        print(end="\n")
     


In [14]:
directory = "../data_lake/"
files = [f for f in os.listdir(directory) if f.endswith(".parquet")]

dfs = {file: pd.read_parquet(directory + file) for file in files}

count = []

for key, value in dfs.items():
    if key.startswith("df_TWB"):
        count.append(str(key.rstrip(".parquet").lstrip("df_TWB_")))
        print(value.head())

print(count)
# dfs["df_UNPD_mort_24.parquet"].loc[
#     :, dfs["df_UNPD_mort_24.parquet"].columns.str.fullmatch("indicator")
# ].mode()

  countryiso3code  date      value unit obs_status  decimal  \
0             AFE  2020  62.121491                        0   
1             AFE  2019  61.813185                        0   
2             AFE  2018  61.470354                        0   
3             AFE  2017  61.083445                        0   
4             AFE  2016  60.643170                        0   

        indicator.id                             indicator.value country.id  \
0  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
1  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
2  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
3  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
4  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   

  country.value  
0                
1                
2                
3                
4                
  countryiso3code  date      value
0  

<IPython.core.display.Javascript object>

In [7]:
UDPD = {
    "df_UNPD_mort_22": "tasa_mortalidad_infantil",
    "df_UNPD_pop_54": "densidad_población_por_kilómetro_cuadrado)",
    "df_UNPD_imigrt_65": "migración_neta_total",
    "df_UNPD_pop_49": "población_total_por_sexo",
    "df_UNPD_mort_60": "total_muertes_por_sexo",
    "df_UNPD_pop_53": "tasa_bruta_cambio_natural_población",
    "df_UNPD_imigrt_66": "tasa_bruta_migración_neta",
    "df_UNPD_pop_72": "proporción_sexos_población_total",
    "df_UNPD_fam_1": "prevalencia_anticonceptivos_porcentaje",
    "df_UNPD_pop_67": "mediana_edad_población",
    "df_UNPD_mort_59": "tasa_bruta_mortalidad_por_1000_habitantes",
    "df_UNPD_pop_51": "tasa_bruta_variación_total_población",
    "df_UNPD_pop_50": "cambio_de_la_población",
    "df_UNPD_pop_41": "población_femenina_edad_reproductiva_(15-49 años)",
    "df_UNPD_mort_24": "tasa_mortalidad_menores_cinco_años",
    "df_UNPD_pop_52": "cambio_natural_población",
    "df_UNPD_fert_19": "tasa_fertilidad",
    "df_UNPD_marstat_42": "estado_civil_casado_porcentaje",
}

<IPython.core.display.Javascript object>

In [8]:
banco_mundial = {
    "SP.DYN.LE00.IN": "esperanza_vida_total",
    "SP.DYN.LE00.FE.IN": "esperanza_vida_mujeres",
    "SP.DYN.LE00.MA.IN": "esperanza_vida_varones",
    "SI.POV.GINI": "índice_gini",
    "SE.XPD.TOTL.GD.ZS": "gasto_púb_educacion_pje",
    "SE.COM.DURS": "duración_educ_obligatoria",
    "NY.GDP.PCAP.CD": "pib_pc_usd_actuales",
    "NY.GDP.MKTP.PP.CD": "pib_ppa_prec_inter",
    "IQ.SCI.OVRL": "capacidad_estadística",
    "SP.POP.TOTL.FE.ZS": "población_mujeres_pje",
    "SP.POP.TOTL.MA.ZS": "población_hombres_pje",
    "NY.GDP.PCAP.PP.CD": "pib_pc_prec_inter",
    "AG.LND.FRST.ZS": "porcentaje_de_bosque",
    "EN.ATM.CO2E.PC": "emisiones_co2",
    "SH.XPD.CHEX.PC.CD": "inversion_salud_percapita",
    "SH.MED.BEDS.ZS": "camas_hospitales_c/1000personas",
    "SP.DYN.IMRT.IN": "mortalidad_infantil_c/1000nacimientos",
    "SH.H2O.BASW.ZS": "acceso_agua_potable(%)",
    "SH.STA.BASS.ZS": "acceso_servicios_sanitarios(%)",
    "SH.STA.SUIC.P5": "tasa_mortalidad_suicidio_c/100.000",
    "SL.UEM.TOTL.ZS": "tasa_desempleo",
    "SP.URB.TOTL.IN.ZS": "tasa_poblacion_urbana",
    "NY.GNP.PCAP.CD": "INB_percapita",
}

<IPython.core.display.Javascript object>

In [15]:
las = [
    "SP.DYN.LE00.MA.IN",
    "SH.STA.BASS.ZS",
    "SI.POV.GINI",
    "SH.XPD.CHEX.PC.CD",
    "NY.GDP.MKTP.PP.CD",
    "SH.STA.SUIC.P5",
    "SE.COM.DURS",
    "SP.DYN.LE00.IN",
    "NY.GDP.PCAP.CD",
    "IQ.SCI.OVRL",
    "SP.POP.TOTL.MA.ZS",
    "NY.GNP.PCAP.CD",
    "SE.XPD.TOTL.GD.ZS",
    "SP.DYN.IMRT.IN",
    "SH.H2O.BASW.ZS",
    "SL.UEM.TOTL.ZS",
    "NY.GDP.PCAP.PP.CD",
    "EN.ATM.CO2E.PC",
    "SP.POP.TOTL.FE.ZS",
    "SH.MED.BEDS.ZS",
]

<IPython.core.display.Javascript object>

In [17]:
banco_mundial = {
    'SP.DYN.LE00.IN': 'esperanza_vida_total',
    'SP.DYN.LE00.FE.IN': 'esperanza_vida_mujeres',
    'SP.DYN.LE00.MA.IN': 'esperanza_vida_varones',
    'SI.POV.GINI': 'índice_gini',
    'SE.XPD.TOTL.GD.ZS': 'gasto_púb_educacion_pje',
    'SE.COM.DURS': 'duración_educ_obligatoria',
    'NY.GDP.PCAP.CD': 'pib_pc_usd_actuales',
    'NY.GDP.MKTP.PP.CD': 'pib_ppa_prec_inter',
    'IQ.SCI.OVRL': 'capacidad_estadística',
    'SP.POP.TOTL.FE.ZS': 'población_mujeres_pje',
    'SP.POP.TOTL.MA.ZS': 'población_hombres_pje',
    'NY.GDP.PCAP.PP.CD': 'pib_pc_prec_inter',
    'AG.LND.FRST.ZS': 'porcentaje_de_bosque',
    'EN.ATM.CO2E.PC': 'emisiones_co2',
    'SH.XPD.CHEX.PC.CD': 'inversion_salud_percapita',
    'SH.MED.BEDS.ZS': 'camas_hospitales_c/1000personas',
    'SP.DYN.IMRT.IN': 'mortalidad_infantil_c/1000nacimientos',
    'SH.H2O.BASW.ZS': 'acceso_agua_potable(%)',
    'SH.STA.BASS.ZS': 'acceso_servicios_sanitarios(%)',
    'SH.STA.SUIC.P5': 'tasa_mortalidad_suicidio_c/100.000',
    'SL.UEM.TOTL.ZS': 'tasa_desempleo',
    'SP.URB.TOTL.IN.ZS': 'tasa_poblacion_urbana',
    'NY.GNP.PCAP.CD': 'INB_percapita'
}

<IPython.core.display.Javascript object>

In [18]:
len(las)

20

<IPython.core.display.Javascript object>

In [19]:
len(banco_mundial)

23

<IPython.core.display.Javascript object>

In [20]:
new_las = []
for key, value in banco_mundial.items():
    new_las.append(key)

print(new_las)

['SP.DYN.LE00.IN', 'SP.DYN.LE00.FE.IN', 'SP.DYN.LE00.MA.IN', 'SI.POV.GINI', 'SE.XPD.TOTL.GD.ZS', 'SE.COM.DURS', 'NY.GDP.PCAP.CD', 'NY.GDP.MKTP.PP.CD', 'IQ.SCI.OVRL', 'SP.POP.TOTL.FE.ZS', 'SP.POP.TOTL.MA.ZS', 'NY.GDP.PCAP.PP.CD', 'AG.LND.FRST.ZS', 'EN.ATM.CO2E.PC', 'SH.XPD.CHEX.PC.CD', 'SH.MED.BEDS.ZS', 'SP.DYN.IMRT.IN', 'SH.H2O.BASW.ZS', 'SH.STA.BASS.ZS', 'SH.STA.SUIC.P5', 'SL.UEM.TOTL.ZS', 'SP.URB.TOTL.IN.ZS', 'NY.GNP.PCAP.CD']


<IPython.core.display.Javascript object>

In [22]:
len(set(las) & set(new_las))

20

<IPython.core.display.Javascript object>

In [26]:
setlist1 = set(las)
setlist2 = set(new_las)
difference1 = setlist2.difference(setlist1)
difference1

{'AG.LND.FRST.ZS', 'SP.DYN.LE00.FE.IN', 'SP.URB.TOTL.IN.ZS'}

<IPython.core.display.Javascript object>